In [66]:
def siguiente_alpha(letra):
    return chr(ord(letra) + 1)

In [67]:
Ruedas = {
    "base": "ABCDEFGHIJKLMNOPQRSTUVWXYZ",
    "I"   : "EKMFLGDQVZNTOWYHXUSPAIBRCJ",
    "II"  : "AJDKSIRUXBLHWTMCQGZNPYFVOE",
    "III" : "BDFHJLCPRTXVZNYEIWGAKMUSQO",
    "IV"  : "ESOVPZJAYQUIRHXLNFTGKDCMWB",
    "V"   : "VZBRGITYUPSDNHLXAWMJQOFECK",
    "VI"  : "JPGVOUMFYQBENHZRDKASXLICTW",
    "VII" : "NZJHGRCXMYSWBOUFAIVLPEKQDT",
    "VIII": "FKQHTLXOCBJSPDZRAMEWNIUYGV"
}

Ruedas_griegas = {
    "base" : "ABCDEFGHIJKLMNOPQRSTUVWXYZ",
    "beta" : "LEYJVCNIXWPBQMDRTAKZGFUHOS",
    "gamma": "FSOKANUERHMBTIYCWLQPZXVGJD"
}

Reflectores = {
    "base": "ABCDEFGHIJKLMNOPQRSTUVWXYZ",
    "A"   : "EJMZALYXVBWFCRQUONTSPIKHGD",
    "B"   : "YRUHQSLDPXNGOKMIEBFZCWVJAT",
    "C"   : "FVPJIAOYEDRZXWGCTKUQSBNMHL"
}

In [107]:
class Rotor:
    def __init__(self, rueda: str, ringstellung: str):      
        self.wiring = rueda
        indice_ring = Ruedas["base"].find(ringstellung)

        # Aplicamos el ringstellung, moviendo cada letra ciertas posiciones con respecto a la base
        wiring_as_str = self.wiring
        
        # Los str son inmutables, así que lo cambiamos temporalmente por una lista
        self.wiring = list(self.wiring)
        
        for i in range(len(self.wiring)):
            indice = Ruedas["base"].find(self.wiring[i]) + indice_ring
            self.wiring[i] = Ruedas["base"][indice % 26]
        

        # Desplazamos la cadena indice_ring lugares a la derecha
        self.rotar(indice_ring)
    
        # Restauramos la cadena
        cadena = ""
        for c in self.wiring:
            cadena = cadena + c
        
        # FIXME
        #self.wiring = cadena   


    def siguiente(self, indice):
        letra_a_buscar = self.wiring[indice]
        indice_siguiente = Ruedas["base"].index(letra_a_buscar)
        return indice_siguiente

    def revertir(self, indice):
        letra_en_base = Ruedas["base"][indice]
        indice_de_salida = self.wiring.index(letra_en_base)    
        return indice_de_salida
    
    def rotar(self, veces: int):
        # Desplazamos la cadena veces lugares a la derecha
        self.wiring = self.wiring[len(self.wiring) - veces : ] + self.wiring[ : len(self.wiring) - veces ]


In [104]:
class Reflector:
    def __init__(self, reflector: str):
        self.wiring = reflector
    
    def reflejar(self, indice):
        """
        `Indice` representa la letra de base[indice].
        """
        letra_a_buscar = self.wiring[indice]
        indice_siguiente = Reflectores["base"].find(letra_a_buscar)
        return indice_siguiente

In [70]:
class Plugboard:
    def __init__(self, parametros):
        """
        El parámetro `mapping` es una lista de duplas, de la forma
        ```
        [(letra1, letra2), (letra3, letra4)]
        ```
        
        De forma que se construirá un mapa tal que letra1 -> letra2, etc...
        """
        
        self.base = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
        self.mapping = {}

        # Nos guardamos primero un mapa de la forma A -> A, B -> B, ...
        for letra in self.base:
            self.mapping[letra] = letra

        # Y ahora lo sustituimos por lo que nos han mandado
        for dupla in parametros:
            self.mapping[dupla[0]] = dupla[1]
            self.mapping[dupla[1]] = dupla[0]
    
    def siguiente(self, letra: str):
        return self.mapping[letra]

In [117]:
class Enigma:
    def __init__(self, ruedas: list[str], ringstellungs: str, grundstellungs: str, reflector: str, steckern: list[(str, str)]):
        if len(ruedas) != len(ringstellungs):
            print("Me has dado un número de ruedas distinto de la configuración del ringstellung")
        if len(ruedas) != len(grundstellungs):
            print("Me has dado un número de ruedas distinto de la configuración del grundstellung")
        
        self.rotores = []
        for rotor, ring in zip(ruedas, ringstellungs):
            self.rotores.append(Rotor(rotor, ring))
        
        self.indices = []
        for letra in grundstellungs:
            self.indices.append(Ruedas["base"].find(letra))
        
        for i, veces in enumerate(self.indices):
            self.rotores[i].rotar(veces)


        self.plugboard = Plugboard(steckern)
        self.reflector = Reflector(reflector)
    
    def cifrar_caracter(self, c: str):
        # Letra -> plugbooard -> rotores -> reflector -> rotores -> plugboard -> salida
        letra = self.plugboard.siguiente(c)
        
        # Aplicar giros
        self.indices[0] = self.indices[0] + 1
        self.rotores[0].rotar(1)

        if self.indices[0] == len(Ruedas["base"]):

            for i in range (1, len(self.indices)):
                self.indices[i-1] = 0

                self.indices[i] = self.indices[i] + 1
                self.rotores[i].rotar(1)

                if i != len(Ruedas["base"]):
                    break
        
        # A partir de aquí, necesitamos los índices, que nos serán más cómodos. Retomaremos la letra más tarde
        indice_letra = Ruedas["base"].find(letra)

        for rotor in self.rotores:
            indice_letra = rotor.siguiente(indice_letra)
        
        indice_letra = self.reflector.reflejar(indice_letra)
    
        for rotor in reversed(self.rotores):
            indice_letra = rotor.revertir(indice_letra)
        
        letra = Ruedas["base"][indice_letra]

        letra = self.plugboard.siguiente(letra)
        
        return letra

    def cifrar(self, mensaje: str):
        # FIXME cuidado con la configuración inicial
        
        salida = ""

        for letra in mensaje:
            if letra == " ":
                salida = salida + " "
            else:
                salida = salida + self.cifrar_caracter(letra)
        
        return salida


In [90]:
def es_conmutativo(mensaje1, mensaje2):
    return mensaje1 == mensaje2

In [118]:
rotores = [Ruedas["V"], Ruedas["VI"], Ruedas["VIII"], Ruedas_griegas["beta"]]
ringstellungs = "EPEL"
grundstellungs = "NAEM"
clavijas = [("A", "E"), ("B", "F"), ("C", "M"), ("D", "Q"), ("H", "U"), ("J", "N"), ("L", "X"), ("P", "R"), ("S", "Z"), ("V", "W")]

enigma = Enigma(rotores, ringstellungs, grundstellungs, Reflectores["C"], clavijas)

mensaje = "DUHF TETO LANO TCTO UARB BFPM HPHG CZXT DYGA HGUF XGEW KBLK GJWL QXXT GPJJ AVTO CKZF SLPP QIHZ FXOE BWII EKFZ LCLO AQJU LJOY HSSM BBGW HZAN VOII PYRB RTDJ QDJJ OQKC XWDN BBTY VXLY TAPG VEAT XSON PNYN QFUD BBHH VWEP YEYD OHNL XKZD NWRH DUWU JUMW WVII WZXI VIUQ DRHY MNCY EFUA PNHO TKHK GDNP SAKN UAGH JZSM JBMH VTRE QEDG XHLZ WIFU SKDQ VELN MIMI THBH DBWV HDFY HJOQ IHOR TDJD BWXE MEAY XGYQ XOHF DMYU XXNO JAZR SGHP LWML RECW WUTL RTTV LBHY OORG LGOW UXNX HMHY FAAC QEKT HSJW DUHF TETO"

nueva_enigma = Enigma(rotores, ringstellungs, grundstellungs, Reflectores["C"], clavijas)

mensaje_cifrado = enigma.cifrar(mensaje)

es_conmutativo(mensaje, nueva_enigma.cifrar(mensaje_cifrado)), mensaje_cifrado

[14, 0, 4, 12]
[15, 0, 4, 12]
[16, 0, 4, 12]
[17, 0, 4, 12]
[18, 0, 4, 12]
[19, 0, 4, 12]
[20, 0, 4, 12]
[21, 0, 4, 12]
[22, 0, 4, 12]
[23, 0, 4, 12]
[24, 0, 4, 12]
[25, 0, 4, 12]
[0, 1, 4, 12]
[1, 1, 4, 12]
[2, 1, 4, 12]
[3, 1, 4, 12]
[4, 1, 4, 12]
[5, 1, 4, 12]
[6, 1, 4, 12]
[7, 1, 4, 12]
[8, 1, 4, 12]
[9, 1, 4, 12]
[10, 1, 4, 12]
[11, 1, 4, 12]
[12, 1, 4, 12]
[13, 1, 4, 12]
[14, 1, 4, 12]
[15, 1, 4, 12]
[16, 1, 4, 12]
[17, 1, 4, 12]
[18, 1, 4, 12]
[19, 1, 4, 12]
[20, 1, 4, 12]
[21, 1, 4, 12]
[22, 1, 4, 12]
[23, 1, 4, 12]
[24, 1, 4, 12]
[25, 1, 4, 12]
[0, 2, 4, 12]
[1, 2, 4, 12]
[2, 2, 4, 12]
[3, 2, 4, 12]
[4, 2, 4, 12]
[5, 2, 4, 12]
[6, 2, 4, 12]
[7, 2, 4, 12]
[8, 2, 4, 12]
[9, 2, 4, 12]
[10, 2, 4, 12]
[11, 2, 4, 12]
[12, 2, 4, 12]
[13, 2, 4, 12]
[14, 2, 4, 12]
[15, 2, 4, 12]
[16, 2, 4, 12]
[17, 2, 4, 12]
[18, 2, 4, 12]
[19, 2, 4, 12]
[20, 2, 4, 12]
[21, 2, 4, 12]
[22, 2, 4, 12]
[23, 2, 4, 12]
[24, 2, 4, 12]
[25, 2, 4, 12]
[0, 3, 4, 12]
[1, 3, 4, 12]
[2, 3, 4, 12]
[3, 3, 4, 12]
[4, 

(True,
 'UAFO JZFL PVMP EDIZ DRJL KQTP DWBW PJZC WUWG SISP AZKB YPXO DHOX PWUE RQOS UARM UHAU EGCK APJL NPIW JRHD BZJT GJXZ USYK TRDI IRQC FINB LIZS ZKVT MUQH QYFA ZWRM QWZW EJFF TDMK UGUI FNNX FTDH DRRW DUCX MRSX XTGR GSVG MRWH AJQD BOSC RMTD EERZ DGDB ORYB OHIZ QNPR AZKL KWAW DPLH FTKU QQIO JCSG NOEI POHK OIQW LHTQ NUKK ZNUW LXZI IPVW CRXH BWGO BTEB RDFY BTSE NPBB TFMR EDQA LCTW TYCL HHVQ RQMS JCDT FPHX DTAT QOCZ CXLL HYTE SDSD ZFYZ MGQF MERL FTHN STDI GJIB VKPV MNQO FXLO OHXJ FVEW PXWF')